In [2]:
import time
import pickle
from collections import Counter
from indra_db import get_primary_db
from indra_db.util import distill_stmts
from indra.literature import pubmed_client
from indra.tools import assemble_corpus as ac


The first step in creating the EMMAA model for Neurofibromatosis is to find publications broadly relevant for neurofibromatosis. We do this by querying based on the both the term "neurofibromatosis" as well as the top-level Medical Subject Heading (MESH) term for Neurofibromatoses, D017253 (see https://www.ncbi.nlm.nih.gov/mesh/68017253).

In [2]:
def get_nf_pmids():
    nf_text_pmids = pubmed_client.get_ids('neurofibromatosis') # PMIDs from text search for "neurofibromatosis"
    nf_mesh_ids = pubmed_client.get_ids_for_mesh('D017253') # Mesh term for neurofibromatoses
    return list(set(nf_text_pmids + nf_mesh_ids))
nf_pmids = get_nf_pmids()

Here's how many articles we found:

In [3]:
len(nf_pmids)

17851

The next step is to get the INDRA Statements that have been extracted from these, which will give us a sense of which molecular entities (proteins, drugs, etc.) are relevant to NF and should be included in the model.

In [4]:
# Get raw statement IDs from the DB for the given TextRefs
reload = False
pmid_stmts_file = 'nf_raw_stmts.pkl'
if reload: 
    db = get_primary_db()
    # Get statements for the given text refs
    print(f"Distilling statements for {len(nf_pmids)} TextRefs")
    start = time.time()
    clauses = [
        db.TextRef.pmid.in_(nf_pmids),
        db.TextContent.text_ref_id == db.TextRef.id,
        db.Reading.text_content_id == db.TextContent.id,
        db.RawStatements.reading_id == db.Reading.id]
    pmid_stmts = distill_stmts(db, get_full_stmts=True, clauses=clauses)
    end = time.time()
    elapsed = end - start
    print(elapsed)
    list(ac.dump_statements(pmid_stmts, pmid_stmts_file))
else:
    pmid_stmts = list(ac.load_statements(pmid_stmts_file))


INFO: [2020-10-28 10:42:57] indra.tools.assemble_corpus - Loading nf_raw_stmts.pkl...
INFO: [2020-10-28 10:42:58] indra.tools.assemble_corpus - Loaded 20209 statements


In [14]:
proteins = [ag.name for stmt in pmid_stmts for ag in stmt.agent_list()
            if ag is not None and 'HGNC' in ag.db_refs]
protein_ctr = Counter(proteins)
protein_ctr = sorted([(k, ct) for k, ct in protein_ctr.items()], key=lambda x: x[1], reverse=True)

In [15]:
protein_ctr

[('NF2', 2843),
 ('NF1', 2665),
 ('EGFR', 292),
 ('MTOR', 276),
 ('DPYSL2', 252),
 ('TP53', 209),
 ('YAP1', 206),
 ('STAT3', 176),
 ('RASA1', 176),
 ('SPRED1', 142),
 ('AKT1', 138),
 ('CAMP', 133),
 ('CTNNB1', 130),
 ('PCNA', 129),
 ('BRAF', 127),
 ('RAC1', 126),
 ('PAK1', 124),
 ('DCAF1', 118),
 ('CDKN1A', 116),
 ('LIMK2', 116),
 ('CD44', 116),
 ('PTK2', 109),
 ('CDC42', 108),
 ('PXN', 106),
 ('LATS1', 102),
 ('TSC2', 98),
 ('HRAS', 97),
 ('ERBB2', 97),
 ('MAP3K11', 95),
 ('RAF1', 90),
 ('KRAS', 80),
 ('ETV5', 76),
 ('EGF', 74),
 ('AMOT', 70),
 ('ALK', 66),
 ('SLC9A3R1', 64),
 ('PTEN', 62),
 ('EZR', 61),
 ('TSC1', 54),
 ('KIT', 54),
 ('GRB10', 54),
 ('VEGFA', 52),
 ('TNFRSF11B', 50),
 ('MLN', 49),
 ('GFER', 49),
 ('LIMK1', 49),
 ('KITLG', 48),
 ('TAZ', 48),
 ('LAYN', 48),
 ('CXCR4', 47),
 ('PDGFRA', 47),
 ('MDM2', 43),
 ('LATS2', 42),
 ('CCL5', 42),
 ('CDKN2A', 40),
 ('NRAS', 39),
 ('EIF4EBP1', 39),
 ('ESR1', 38),
 ('NRG1', 37),
 ('PEBP1', 37),
 ('GNL3', 36),
 ('TNFSF10', 36),
 ('SGK1

In [16]:
ag_stmts = [s for s in pmid_stmts if 'DCAF1' in [ag.name for ag in s.agent_list() if ag is not None]]

In [18]:
ag_stmts.sort(key=lambda x: len(x.evidence), reverse=True)

In [19]:
from indra.sources import indra_db_rest as idr

In [33]:
idrp = idr.get_statements_for_paper([('pmid', pmid) for pmid in nf_pmids][0:100])

INFO: [2020-10-27 10:28:54] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_papers
INFO: [2020-10-27 10:28:54] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'max_stmts': None, 'api_key': '[api-key]'}


Get drugs that:
1. Have already been tested against NF (are in the screening datasets)
2. Are known to target one or more genes relevant to NF.

In [ ]:
both_pmids = set(nf_mesh_ids).union(set(nf_text_pmids))

# Ras Model

In [8]:
with open('model_2019-04-01-14-36-38.pkl', 'rb') as f:
    emmaa_stmts = pickle.load(f)
ras_stmts = [es.stmt for es in emmaa_stmts]

In [9]:
ras_stmts

[Complex(IGF1R(), IGF1R()),
 Autophosphorylation(IGF1R(bound: [IGF1R, True]), Y, 1166),
 ActiveForm(IGF1R(mods: (phosphorylation, Y, 1166)), activity, True),
 Phosphorylation(IGF1R(activity), IRS1(), Y),
 ActiveForm(IRS1(mods: (phosphorylation, Y)), activity, True),
 Complex(IRS1(activity), PIK3CA()),
 DecreaseAmount(None, IRS1(mods: (phosphorylation, S, 312))),
 Dephosphorylation(PPP2CA(activity), IRS1(), S, 312),
 Phosphorylation(RPS6KB1(), IRS1(), S, 312),
 Complex(EGF(), EGFR()),
 Complex(EGFR(bound: [EGF, True]), EGFR(bound: [EGF, True])),
 Autophosphorylation(EGFR(bound: [EGFR, True]), Y, 1016),
 Autophosphorylation(EGFR(bound: [EGFR, True]), Y, 1069),
 Autophosphorylation(EGFR(bound: [EGFR, True]), Y, 1092),
 Autophosphorylation(EGFR(bound: [EGFR, True]), Y, 1110),
 Autophosphorylation(EGFR(bound: [EGFR, True]), Y, 1138),
 Autophosphorylation(EGFR(bound: [EGFR, True]), Y, 1172),
 Autophosphorylation(EGFR(bound: [EGFR, True]), Y, 1197),
 Phosphorylation(EGFR(bound: [EGFR, True]),